<h2> 3.Explore and Cluster the neighborhoods in Toronto</h2>

Just make sure:
1) to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
2) to generate maps to visualize your neighborhoods and how they cluster together. 

In [1]:
#import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

# creation of a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

#data cleaning
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#display cleaned dataframe
df=pd.DataFrame(table_contents)
df.shape
df.head(12)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
#defininf get_geocode funciton
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

#import location data
geo_df=pd.read_csv('C:\\Users\Lucio\Desktop\IBM - Applied Data Science Capstone\Geospatial_Coordinates.csv')

#rename geo_df columns and merge dataframes
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_merged = geo_df.merge(df, on='PostalCode')

#reordering columns
geo_data = df_merged[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
#filtering the data for Toronto
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data =toronto_data[df['Borough'].str.split().str.len().lt(2)]
toronto_data

<ipython-input-4-56fede8980a2>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  toronto_data =toronto_data[df['Borough'].str.split().str.len().lt(2)]


,PostalCode,Borough,Neighborhood,Latitude,Longitude
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
58,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
64,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
70,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
77,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
78,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191


In [5]:
#Foursquare ID
CLIENT_ID = 'F2NIXHCXKDNEUOI45KWOAYS4MJ3XXMYRLWSYHWFURXLVSC4I'
#Foursquare Secret
CLIENT_SECRET = 'LKF224VRCLQNOCGVMSQWBOVWQOGHBPXC2BK5HQBP2HRWEC0U' 

VERSION = '20180604'

In [6]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Lawrence Park
St. James Town, Cabbagetown
Berczy Park
Richmond, Adelaide, King
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
First Canadian Place, Underground city
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South


In [8]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,"St. James Town, Cabbagetown",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [9]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"First Canadian Place, Underground city",100,100,100,100,100,100
Forest Hill North & West,4,4,4,4,4,4
"High Park, The Junction South",24,24,24,24,24,24
Lawrence Park,3,3,3,3,3,3
"Little Portugal, Trinity",42,42,42,42,42,42
"Richmond, Adelaide, King",93,93,93,93,93,93
Roselawn,2,2,2,2,2,2


In [10]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(415, 144)

In [11]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.000000,0.017544,0.0,0.000000,0.00,0.0,0.017544,0.035088,...,0.0,0.00,0.017544,0.00,0.00,0.00,0.017544,0.0,0.00,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.086957,...,0.0,0.00,0.000000,0.00,0.00,0.00,0.000000,0.0,0.00,0.0
2,"First Canadian Place, Underground city",0.03,0.000000,0.010000,0.0,0.000000,0.03,0.0,0.000000,0.010000,...,0.0,0.01,0.020000,0.01,0.00,0.01,0.010000,0.0,0.01,0.0
3,Forest Hill North & West,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,...,0.0,0.00,0.000000,0.00,0.25,0.00,0.000000,0.0,0.00,0.0
4,"High Park, The Junction South",0.00,0.041667,0.000000,0.0,0.041667,0.00,0.0,0.000000,0.041667,...,0.0,0.00,0.083333,0.00,0.00,0.00,0.000000,0.0,0.00,0.0


In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Pharmacy,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Beer Bar,Bakery,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Grocery Store,Furniture / Home Store,Gym,Intersection,Italian Restaurant,Convenience Store
2,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,American Restaurant,Asian Restaurant,Salad Place,Seafood Restaurant
3,Forest Hill North & West,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Donut Shop,Discount Store,Diner,Department Store
4,"High Park, The Junction South",Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Bar,Cajun / Creole Restaurant,Fast Food Restaurant,Bookstore,Flea Market,Speakeasy


<h3> Clustering neighborhood

In [15]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 0, 1, 3, 1, 1, 2, 1])

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Eastern European Restaurant,Donut Shop,Discount Store,Diner,Department Store,Deli / Bodega,Cupcake Shop
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Café,Pizza Place,Coffee Shop,Italian Restaurant,Park,Chinese Restaurant,Market,Restaurant,Bakery,Pub
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Pharmacy,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Beer Bar,Bakery,Concert Hall
58,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Deli / Bodega,Gym,Thai Restaurant,Clothing Store,American Restaurant,Salad Place,Cosmetics Shop
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Home Service,Garden,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner,Department Store


In [17]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Coffee Shop,Cocktail Bar,Pharmacy,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Beer Bar,Bakery,Concert Hall
1,1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Grocery Store,Furniture / Home Store,Gym,Intersection,Italian Restaurant,Convenience Store
2,1,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,American Restaurant,Asian Restaurant,Salad Place,Seafood Restaurant
3,0,Forest Hill North & West,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Donut Shop,Discount Store,Diner,Department Store
4,1,"High Park, The Junction South",Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Bar,Cajun / Creole Restaurant,Fast Food Restaurant,Bookstore,Flea Market,Speakeasy


In [18]:

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


<h3>Creating MAP

In [19]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters